In [42]:
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import sympy as sp
sp.init_printing()

In [43]:
# a = 1
# c = 1
# d = 5
# x_rest = -1.6
b = 0.5
# r = 0.01
# s = 4
I = 1.5

In [44]:
f1 = lambda x, y, z,  b, I: y - x**3 + b*x**2 + I - z
f2 = lambda x, y: 1 - 5*x**2 - y
f3 = lambda x, z: 0.01*(4*(x + 1.6) - z)

In [56]:
S = lambda x, y, z, b, I: y - x**3 + b*x**2 + I - z

dSdx = lambda x, b: -3*x**2 + 2*b*x
dSdy = lambda: 1
dSdz = lambda: -1

def H (x, y, z, b, I): 
    F1 = f1(x, y, z,  b, I)
    F2 = f2(x, y)
    F3 = f3(x, z)
    
    dS_dx = dSdx(x, b)
    
    return dS_dx*F1 + F2 - F3

In [46]:
dxdS = lambda x, y, z, b, I: f1(x, y, z, b, I)/H(x, y, z, b, I)
dydS = lambda x, y, z, b, I: f2(x, y)/H(x, y, z, b, I)
dzdS = lambda x, y, z, b, I: f3(x, z,)/H(x, y, z, b, I)
dtdS_non_standart = lambda x, y, z, b, I: 1/H(x, y, z, b, I)
dtdS_standart = lambda: 1

In [60]:
def RK4_standart_step (S_start, x_start, y_start, z_start, t_start, step, b, I):

    S_next = S_start + step;

    k1x = dxdS(x_start, y_start, z_start, b, I)
    k1y = dydS(x_start, y_start, z_start, b, I)
    k1z = dzdS(x_start, y_start, z_start, b, I)
    k1t = dtdS_standart()

    k2x = dxdS(x_start + 0.5*step*k1x, y_start + 0.5*step*k1x, z_start + 0.5*step*k1x, b, I)
    k2y = dydS(x_start + 0.5*step*k1y, y_start + 0.5*step*k1y, z_start + 0.5*step*k1y, b, I)
    k2z = dzdS(x_start + 0.5*step*k1z, y_start + 0.5*step*k1z, z_start + 0.5*step*k1z, b, I)
    k2t = dtdS_standart()

    k3x = dxdS(x_start + 0.5*step*k2x, y_start + 0.5*step*k2x, z_start + 0.5*step*k2x, b, I)
    k3y = dydS(x_start + 0.5*step*k2y, y_start + 0.5*step*k2y, z_start + 0.5*step*k2y, b, I)
    k3z = dzdS(x_start + 0.5*step*k2z, y_start + 0.5*step*k2z, z_start + 0.5*step*k2z, b, I)
    k3t = dtdS_standart()

    k4x = dxdS(x_start + step*k3x, y_start + step*k3x, z_start + step*k3x, b, I)
    k4y = dydS(x_start + step*k3y, y_start + step*k3y, z_start + step*k3y, b, I)
    k4z = dzdS(x_start + step*k3z, y_start + step*k3z, z_start + step*k3z, b, I)
    k4t = dtdS_standart()

    x_next = x_start + (step/6)*(k1x + 2*k2x + 2*k3x + k4x)
    y_next = y_start + (step/6)*(k1y + 2*k2y + 2*k3y + k4y)
    z_next = x_start + (step/6)*(k1z + 2*k2z + 2*k3z + k4z)
    t_next = x_start + (step/6)*(k1t + 2*k2t + 2*k3t + k4t)

    return [S_next, x_next, y_next, z_next, t_next]

In [48]:
def RK4_non_standart_step (S_start, x_start, y_start, z_start, t_start, step, b, I):

    S_next = S_start + step;

    k1x = dxdS(x_start, y_start, z_start, b, I)
    k1y = dydS(x_start, y_start, z_start, b, I)
    k1z = dzdS(x_start, y_start, z_start, b, I)
    k1t = dtdS_non_standart(x_start, y_start, z_start, b, I)

    k2x = dxdS(x_start + 0.5*step*k1x, y_start + 0.5*step*k1x, z_start + 0.5*step*k1x, b, I)
    k2y = dydS(x_start + 0.5*step*k1y, y_start + 0.5*step*k1y, z_start + 0.5*step*k1y, b, I)
    k2z = dzdS(x_start + 0.5*step*k1z, y_start + 0.5*step*k1z, z_start + 0.5*step*k1z, b, I)
    k2t = dtdS_non_standart(x_start + 0.5*step*k1t, y_start + 0.5*step*k1t, z_start + 0.5*step*k1t, b, I)

    k3x = dxdS(x_start + 0.5*step*k2x, y_start + 0.5*step*k2x, z_start + 0.5*step*k2x, b, I)
    k3y = dydS(x_start + 0.5*step*k2y, y_start + 0.5*step*k2y, z_start + 0.5*step*k2y, b, I)
    k3z = dzdS(x_start + 0.5*step*k2z, y_start + 0.5*step*k2z, z_start + 0.5*step*k2z, b, I)
    k3t = dtdS_non_standart(x_start + 0.5*step*k2t, y_start + 0.5*step*k2t, z_start + 0.5*step*k2t, b, I)

    k4x = dxdS(x_start + step*k3x, y_start + step*k3x, z_start + step*k3x, b, I)
    k4y = dydS(x_start + step*k3y, y_start + step*k3y, z_start + step*k3y, b, I)
    k4z = dzdS(x_start + step*k3z, y_start + step*k3z, z_start + step*k3z, b, I)
    k4t = dtdS_non_standart(x_start + step*k3t, y_start + step*k3t, z_start + step*k3t, b, I)

    x_next = x_start + (step/6)*(k1x + 2*k2x + 2*k3x + k4x)
    y_next = y_start + (step/6)*(k1y + 2*k2y + 2*k3y + k4y)
    z_next = x_start + (step/6)*(k1z + 2*k2z + 2*k3z + k4z)
    t_next = x_start + (step/6)*(k1t + 2*k2t + 2*k3t + k4t)

    return [S_next, x_next, y_next, z_next, t_next]

In [78]:
def Henon (S_min, S_max, x_start, y_start, z_start, t_start, step, b, I):

    X = []
    Y = []
    Z = []
    T = []
    S_vals = []

    x = x_start
    y = y_start
    z = z_start
    t = t_start
    S_start = S_min
    next_step = step

    while (S_start < S_max):
        S_prev = S(x, y, z, b, I)
#         print(S_prev)
        new_point = RK4_standart_step(S_start, x, y, z, t, next_step, b, I)
        S_start, x, y, z, t = new_point;
        S_next = S(x, y, z, b, I);
#         print(S_next)

        if S_prev * S_next < 0:
            print(S_prev)
            print(S_next)
            next_step = -S_next
            print(next_step)
            new_point = RK4_non_standart_step(S_start, x, y, z, t, next_step, b, I)
            print(new_point)
            S_start, x, y, z, t = new_point;
            print(x)
            print(y)
            print(z)
            S_test = S(x, y, z, b, I)
            print(S_test)
            S_vals.append(S_start)
            X.append(x)
            Y.append(y)
            Z.append(z)
            T.append(t)

    return[X, Y, Z, T, S_vals]

In [79]:
Henon(0, 100, 0, 0, 0, 0, 0.01, b, I)

1.807987989560845
-10.398375883031289
10.398375883031289
[10.888375883031289, 10.552283776623202, 2.6429813483960327, 2.4843413998887045, 2.5423411013113455]
10.552283776623202
2.6429813483960327
2.4843413998887045
-1117.6701238298922
